In [ ]:

import numpy as np
import pandas as pd
import cv2 as cv
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

plt.style.use("seaborn")


In [ ]:

# Load image and apply advanced image processing techniques
def load_processed_image(path):
    image = cv.imread(path, 0)
    # Applying CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    processed_image = clahe.apply(image)
    plt.imshow(cv.cvtColor(processed_image, cv.COLOR_BGR2RGB))
    plt.show()
    return processed_image

image_path = 'line1.jpg'
image = load_processed_image(image_path)


In [ ]:

# Advanced image analysis using contours
def analyze_image(image):
    contours, _ = cv.findContours(image, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    image_contours = cv.drawContours(image.copy(), contours, -1, (0, 255, 0), 3)
    plt.imshow(image_contours)
    plt.show()
    return contours

contours = analyze_image(image)


In [ ]:

# Data extraction for machine learning from contour analysis
def extract_features(contours):
    features = [cv.contourArea(cnt) for cnt in contours]
    return features

features = extract_features(contours)
labels = np.array([1 if f > 100 else 0 for f in features])  # Label based on contour area

x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.3, random_state=42)

# Train Logistic Regression
logistic_model = LogisticRegression()
logistic_model.fit(np.array(x_train).reshape(-1, 1), y_train)
